# Agent

Lets whip up a quick langgraph agent that loops with a single tool.

In [1]:
# Reload
%load_ext autoreload
%autoreload 2


In [12]:
%pip install -q langgraph langchain-openai langchain-core



Note: you may need to restart the kernel to use updated packages.


In [2]:
from utils.data_collection import load_df, MovieSearchTool

# df = load_df("../raw/top_50000.pkl")


In [3]:
movie_data_tool = MovieSearchTool()


Loading df


In [4]:
movie_data_tool.semantic_search("Matrix", k=3)


Got query embeddings


,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,...,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,embedding,embedding_norm,dot_product,similarity
15,The Matrix,8.217,25416.0,Released,1999-03-31,463517383.0,136.0,63000000.0,tt0133093,en,...,Bill Pope,"Lana Wachowski, Lilly Wachowski","Erwin Stoff, Barrie M. Osborne, Joel Silver, A...",Don Davis,8.7,2088357.0,"[0.010932380333542824, 0.009856754913926125, -...",1.0,0.442468,0.442468
263,The Matrix Reloaded,7.054,10827.0,Released,2003-05-15,738599701.0,138.0,150000000.0,tt0234215,en,...,Bill Pope,"Lana Wachowski, Lilly Wachowski","Grant Hill, Andrew Mason, Joel Silver, Bruce B...",Don Davis,7.2,638656.0,"[0.0051968153566122055, -0.01295418106019497, ...",1.0,0.436334,0.436334
43500,Matrix,6.904,26.0,Released,1998-12-31,0.0,7.0,0.0,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,"[0.023806946352124214, 0.008451313711702824, -...",1.0,0.546798,0.546798


In [14]:
from utils.langgraph import make_graph

graph = make_graph([movie_data_tool])

In [16]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
results = graph.invoke(input={"messages": [HumanMessage(content="When was the matrix reloaded made?")]})
results["messages"][-1].content

Got query embeddings


'"The Matrix Reloaded" was released on May 15, 2003. You can find more information about the movie [here](https://www.imdb.com/title/tt0234215).'

In [8]:
results["messages"][-1]

ToolMessage(content='Error: TypeError("MovieData.semantic_search_tool() missing 1 required positional argument: \'self\'")\n Please fix your mistakes.', name='movie_data_search', id='fbce226b-9fef-4db5-b3b9-44c0a9feb8d5', tool_call_id='call_J7ae7SI0NrsJtxtrm6BfsU59')

In [9]:

messages = results["messages"]
# Remove last message
messages = messages[:-1]

messages


[HumanMessage(content='When was the matrix reloaded made?', additional_kwargs={}, response_metadata={}, id='0f98e6ee-3997-48a1-ae82-2aa4ddd2c27e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_J7ae7SI0NrsJtxtrm6BfsU59', 'function': {'arguments': '{"query":"The Matrix Reloaded"}', 'name': 'movie_data_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 77, 'total_tokens': 95, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-941f7642-64a3-4199-bfee-076638e95ca7-0', tool_calls=[{'name': 'movie_data_search', 'args': {'query': 'The Matrix Reloaded'}, 'id': 'call_J7ae7SI0NrsJtxtrm6BfsU59', 'type': 'tool_call'}], usage_metadata={'input_tokens': 77, 'output_tokens': 18, 'total_tokens': 95})]